# Test `pyimagesearch` object

Objectives: play around with and test the objects defined in the modules within the `pyimagesearch/` directory.

In [ ]:
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../pyimagesearch/')

sns.set_theme()

%load_ext autoreload
%autoreload 2

## `dataset.py`

**Note:** make sure to download (and point to) the correct file!

In [ ]:
from tensorflow.keras.layers import TextVectorization
from dataset import load_data, splitting_dataset, make_dataset, tf_lower_and_split_punct

In [ ]:
fname = '../data/fra.txt'

source, target = load_data(fname)

print('Some source sentences:', source[:3])
print('Some target sentences:', target[:3])

In [ ]:
(
    (trainSource, trainTarget),
    (valSource, valTarget),
    (testSource, testTarget)
) = splitting_dataset(source, target)

In [ ]:
test_dataset = make_dataset(
    splits=(trainSource, trainTarget),
    batchSize=16,
    # The text processors should probably be instances
    # with particular initializations (depending on the
    # source and target vocabularies etc.). Here it's
    # sufficient the function runs correctly.
    sourceTextProcessor=TextVectorization(),
    targetTextProcessor=TextVectorization(),
    train=False
)

test_dataset

Work on a test sentences with the text manipulation function.

In [ ]:
test_sentences = [
    "Six o'clock on the Christmas morning...",
    "and for what?",
    "Six o'clock the siren kicks him from a dream",
    "Tries to shake it off but it just won't stop"
]

processed_test_sentences = tf_lower_and_split_punct(test_sentences)

processed_test_sentences

## `attention.py`

From the Keras documentation: layers (and any subclass) are **callable objects** in which operations are implemented in their **call** method. So:
- To act with a layer on some input, just pass the input to it as you would to a function.
- To define what the layer does, look at its **call** method.

In [ ]:
from attention import BaseAttention, CrossAttention

In [ ]:
x_test = tf.random.uniform(shape=(10, 100))
context_test = tf.random.uniform(shape=(50, 100))

In [ ]:
bal = BaseAttention(num_heads=4, key_dim=10)

bal(tf.concat([x_test, context_test], axis=0))

## `positional_encoding.py`

In [ ]:
from positional_encoding import positional_encoding, PositionalEmbedding
from tensorflow.keras.layers import TextVectorization

In [ ]:
test_lenght = 50  # Max length of a sequence.
test_depth = 50  # Length of the representation of each position.

pe = positional_encoding(length=test_lenght, depth=test_depth)

pe

Transform the processed test sentences (still in natural language) into a numerical tensor via a `TextVectorization` layer, then get its embedding.

In [ ]:
from itertools import chain
import numpy as np

In [ ]:
test_vocabulary = np.unique(
    list(chain.from_iterable([s.numpy().decode('utf-8').split(' ') for s in processed_test_sentences]))
)

tvl = TextVectorization(
    # We extract all the unique tokens from the processed sentences and
    # use them as the vocabulary.
    vocabulary=test_vocabulary
)

vectorized_sentences = tvl(processed_test_sentences)

vectorized_sentences

In [ ]:
# Compare with the original processed sentences in natural
# language.
processed_test_sentences

In [ ]:
dModel = 50

test_vocab_size = test_vocabulary.shape[0] + 2
test_max_pos_encoding = 50

pos_emb = PositionalEmbedding(
    # We must add at least 2 to the vocebulary size for this
    # to work, probably because of special tokens like
    # padding with zeros or start/end sentence tokens.
    vocabSize=test_vocab_size,
    dModel=dModel,
    maximumPositionEncoding=test_max_pos_encoding
)

In [ ]:
fig = plt.figure(figsize=(14, 6))

plt.imshow(pos_emb.posEncoding)

In [ ]:
# Get the embedding with positional encoding information for
# the vectorized sentences.
test_embeddings = pos_emb(vectorized_sentences)

test_embeddings

## `feedforward.py`

In [ ]:
from feed_forward import FeedForward

Process the test embeddings with the custom `FeedForward` layer. Notice how the input and output shape do not change: that's needed for the skip connection (adding up the input tensor back to the output one).

In [ ]:
ff = FeedForward(
    dff=128,
    dModel=dModel
)

print(f'Test embeddings shape: {test_embeddings.shape}')

ff(test_embeddings)

## `rate_schedule.py`

In [ ]:
from rate_schedule import CustomSchedule

In [ ]:
cs = CustomSchedule(dModel=dModel, warmupSteps=10)

In [ ]:
steps = tf.range(1, 100)

schedule = cs(steps)

schedule

In [ ]:
fig = plt.figure(figsize=(14, 6))

sns.lineplot(
    x=steps,
    y=schedule
)

plt.xlabel('Step')
plt.ylabel('Learning rate')

## `loss_accuracy.py`

In [ ]:
from loss_accuracy import masked_loss, masked_accuracy

Generate fake labels and predictions and compute the masked loss and accuracy over them.

**Note:** shapes and number of labels are chosen so that the loss and accuracy are computed correctly. The requirement is that since the predictions are logits, each prediction is a tensor with a shape equal to the number of labels. Also, "middle" shape of the predictions tensor has been chosen so as to make both the loss and the accuracy work (the accuracy looks for axis 2 when computing the argmax).

In [ ]:
n_samples = 50

test_labels = tf.random.uniform(shape=[n_samples, 1], minval=0, maxval=10, dtype=tf.int32)

test_predictions = tf.random.uniform(shape=[n_samples, test_vocabulary.shape[0] + 2])
test_predictions = test_predictions / tf.reduce_sum(test_predictions, axis=-1)[..., tf.newaxis]
test_predictions = test_predictions[:, tf.newaxis, :]

print('Masked loss:', masked_loss(test_labels, test_predictions))
print('Masked accuracy:', masked_accuracy(test_labels, test_predictions))

## `encoder.py`

In [ ]:
from encoder import EncoderLayer, Encoder

In [ ]:
encoder = Encoder(
    numLayers=4,
    dModel=dModel,
    numHeads=4,
    sourceVocabSize=test_vocab_size,
    maximumPositionEncoding=test_max_pos_encoding,
    dff=512,
)

Passing the vectorized sentences to the encoder (with the correct dimensions found before) should work!

In [ ]:
encoder(vectorized_sentences)

## `decoder.py`

In [ ]:
from decoder import DecoderLayer, Decoder

In [ ]:
target_test_sentences = [
    "Sei del mattino la mattina di Natale...",
    "e per cosa?",
    "Sei del mattino la sirena lo sveglia da un sogno",
    "Cerca di scrollarselo via ma proprio non la smette"
]

processed_target_test_sentences = tf_lower_and_split_punct(target_test_sentences)

processed_target_test_sentences

In [ ]:
target_test_vocabulary = np.unique(
    list(chain.from_iterable([s.numpy().decode('utf-8').split(' ') for s in processed_target_test_sentences]))
)

tvl_target = TextVectorization(
    # We extract all the unique tokens from the processed sentences and
    # use them as the vocabulary.
    vocabulary=target_test_vocabulary
)

vectorized_target_sentences = tvl_target(processed_target_test_sentences)

vectorized_target_sentences

In [ ]:
target_test_vocab_size = target_test_vocabulary.shape[0] + 2

decoder = Decoder(
    numLayers=4,
    dModel=dModel,
    numHeads=4,
    targetVocabSize=target_test_vocab_size,
    maximumPositionEncoding=test_max_pos_encoding,
    dff=512
)

## `transformer.py`

In [ ]:
from transformer import Transformer

In [ ]:
trns = Transformer(
    encNumLayers=4,
    decNumLayers=4,
    dModel=dModel,
    numHeads=4,
    dff=512,
    sourceVocabSize=test_vocab_size,
    targetVocabSize=target_test_vocab_size,
    maximumPositionEncoding=test_max_pos_encoding
)

In [ ]:
trns(inputs=(vectorized_sentences, vectorized_target_sentences))

## `translator.py`

Test the the `Translator` object is at least initialized without errors: for the actual translation more details must be known from the various objects that come with it (e.g. the text processors).

In [ ]:
from translate import Translator

In [ ]:
test_translator = Translator(
    sourceTextProcessor=tvl,
    targetTextProcessor=tvl_target,
    transformer=trns,
    maxLength=20
)